In [4]:
from config import tushare_api_key
import pandas as pd
import tushare as ts
ts.set_token(tushare_api_key)
pro = ts.pro_api()

In [12]:
# Fetching adjusted ('除权') historical data for 000001.SZ since its IPO by year
# Define the stock code
stock_code = '000001.SZ'

# Initialize an empty DataFrame to hold the concatenated results
df_full = pd.DataFrame()

# Define the start date for fetching data
start_date = '19910101'

# Define the end date as today
end_date = pd.Timestamp.today().strftime('%Y%m%d')

# Convert start_date and end_date to datetime objects
start_date_dt = pd.to_datetime(start_date)
end_date_dt = pd.to_datetime(end_date)

# Calculate the number of years between start and end date
years = end_date_dt.year - start_date_dt.year + 1

# Fetch the adjusted historical data in yearly batches
for year in range(start_date_dt.year, start_date_dt.year + years):
    # Define yearly start and end dates
    yearly_start_date = f"{year}0101"
    yearly_end_date = f"{year}1231"
    # Ensure the end date does not exceed the current date
    if pd.to_datetime(yearly_end_date) > end_date_dt:
        yearly_end_date = end_date

    df_batch = pro.daily(ts_code=stock_code, adj='hfq', start_date=yearly_start_date, end_date=yearly_end_date)
    # Adjust the order of the fetched data before concatenation
    df_batch = df_batch.sort_values(by='trade_date', ascending=True)
    df_full = pd.concat([df_full, df_batch], ignore_index=True)

# Selecting required columns: 'trade_date', 'vol', 'open', 'high', 'low', 'close'
df_selected = df_full[['trade_date', 'vol', 'open', 'high', 'low', 'close']]

# Display the first few rows of the dataframe
print(df_selected, end='')


     trade_date         vol   open   high    low  close
0      19910404        3.00  48.76  48.76  48.76  48.76
1      19910405        2.00  48.52  48.52  48.52  48.52
2      19910408        2.00  48.04  48.04  48.04  48.04
3      19910409        4.00  47.80  47.80  47.80  47.80
4      19910410       15.00  47.56  47.56  47.56  47.56
...         ...         ...    ...    ...    ...    ...
7834   20240326  1740021.46  10.45  10.66  10.42  10.60
7835   20240327  1274135.99  10.56  10.63  10.51  10.53
7836   20240328  1302188.92  10.51  10.57  10.38  10.49
7837   20240329   872758.98  10.45  10.57  10.43  10.52
7838   20240401  1191087.96  10.52  10.65  10.51  10.64

[7839 rows x 6 columns]